# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0,'..')

from src.base.data_loaders.data_loader import DLName
from src.base.gt_loaders.gt_names import GTName
from src.exp_runner import ExperimentRunner
from src.base.experiment.base_models import BaseModel
from src.base.experiment.benchmark_dataset import BenchmarkDataset
from src.base.experiment.model_creator import Optimizer
from src.base.experiment.model_evaluator import DataSource, DataPredSelection
from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.constants import ICAO_REQ, MNIST_TASK

 ==> Restrict GPU memory growth: True


# Start Network runner

In [3]:
kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'train_vg16_mtl_3_approach',
        'description': 'Training MTL network for FVC-ICOA tasks with Handcrafted 3 network architecture',
        'tags': ['mtl', 'handcrafted_3', 'handcrafted_3_exp', 'fvc', 'ground_truths', '200 epochs'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': MTLApproach.HAND_3,
        'benchmarking': {
            'use_benchmark_data': False,
            'benchmark_dataset': BenchmarkDataset.MNIST,
            'tasks': list(MNIST_TASK)
        },
        'icao_data': {
            'icao_gt': {
                'use_gt_data': True,
                'gt_names': {
                    'train_validation': [],
                    'test': [],
                    'train_validation_test': [GTName.FVC]
                },
            },
            'icao_dl': {
                'use_dl_data': False,
                'tagger_model': None
            },
            'reqs': list(ICAO_REQ),
            'aligned': False,
        },
        'balance_input_data': False,
        'train_model': False,
        'save_trained_model': False,
        'exec_nas': False,
        'orig_model_experiment_id': 'ICAO-338',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 32,
        'n_epochs': 200,
        'early_stopping': 200,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {}
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 17689
Process ID: 360211
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'exp_params': {'description': 'Training MTL network for FVC-ICOA tasks with '
                               'Handcrafted 3 network architecture',
                'name': 'train_vg16_mtl_3_approach',
                'src_files': ['../src/**/*.py'],
                'tags': ['mtl',
                         'handcrafted_3',
                         'handcrafted_3_exp',
                         'fvc',
                         'ground_truths',
                         '200 epochs']},
 'nas_params': {},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'name': 'vgg16', 'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f2d04517280>}>,
                      'batch_size': 32,
                      'dropout': 0.3,
                      'early_sto

# Load Data

In [4]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4928, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


# Sampling Training Data

In [5]:
runner.sample_training_data()

-------------------- sample training data -------------------
Not applying subsampling in training data!


# Data Balancing

In [6]:
runner.balance_input_data()

-------------------- balance input data -------------------
Not balancing input_data


# Data Generators

In [7]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 4928 validated image filenames.
Found 547 validated image filenames.
Found 288 validated image filenames.
TOTAL: 5763

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Setup Experiment

In [8]:
runner.setup_experiment()

-------------------- create experiment -------------------
Setup neptune properties and parameters
Properties and parameters setup done!


# Labels Distribution

In [9]:
runner.summary_labels_dist()

-------------------- summary labels dist -------------------
Requisite: MOUTH
N_TRAIN_COMP: 3607 (73.19%)
N_TRAIN_NOT_COMP: 1321 (26.81%)
N_TRAIN_DUMMY: 0 (0.0%)
N_VALIDATION_COMP: 420 (76.78%)
N_VALIDATION_NOT_COMP: 127 (23.22%)
N_VALIDATION_DUMMY: 0 (0.0%)
N_TEST_COMP: 215 (74.65%)
N_TEST_NOT_COMP: 73 (25.35%)
N_TEST_DUMMY: 0 (0.0%)
----
Requisite: ROTATION
N_TRAIN_COMP: 3716 (75.41%)
N_TRAIN_NOT_COMP: 1212 (24.59%)
N_TRAIN_DUMMY: 0 (0.0%)
N_VALIDATION_COMP: 410 (74.95%)
N_VALIDATION_NOT_COMP: 137 (25.05%)
N_VALIDATION_DUMMY: 0 (0.0%)
N_TEST_COMP: 208 (72.22%)
N_TEST_NOT_COMP: 80 (27.78%)
N_TEST_DUMMY: 0 (0.0%)
----
Requisite: L_AWAY
N_TRAIN_COMP: 3375 (68.49%)
N_TRAIN_NOT_COMP: 1553 (31.51%)
N_TRAIN_DUMMY: 0 (0.0%)
N_VALIDATION_COMP: 382 (69.84%)
N_VALIDATION_NOT_COMP: 165 (30.16%)
N_VALIDATION_DUMMY: 0 (0.0%)
N_TEST_COMP: 201 (69.79%)
N_TEST_NOT_COMP: 87 (30.21%)
N_TEST_DUMMY: 0 (0.0%)
----
Requisite: EYES_CLOSED
N_TRAIN_COMP: 3887 (78.88%)
N_TRAIN_NOT_COMP: 1041 (21.12%)
N_TRAIN_D

# Create Model

In [10]:
runner.create_model()

-------------------- create model -------------------
Not creating a model: not training a model! 


# Vizualize Model

In [11]:
runner.visualize_model(outfile_path=f"figs/handcrafted_mtl_model_3.png")

-------------------- vizualize model -------------------
Not training a model! No model to exhibit!


In [12]:
runner.model_summary()

Not training a model!


# Training Model

In [13]:
runner.train_model()

-------------------- train model -------------------
Not training a model. Downloading data from Neptune
 ..Experiment ID: ICAO-338
 ..Downloading data from previous experiment
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-338
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
 ..Requisite: ICAO_REQ.MOUTH
 ..Download finished
 ..Upload data to current experiment
 ..Requisite: ICAO_REQ.ROTATION
 ..Download finished
 ..Upload data to current experiment
 ..Requisite: ICAO_REQ.L_AWAY
 ..Download finished
 ..Upload data to current experiment
 ..Requisite: ICAO_REQ.EYES_CLOSED
 ..Download finished
 ..Upload data to current experiment
 ..Requisite: ICAO_REQ.CLOSE
 ..Download finished
 ..Upload data to current experiment
 ..Requisite: ICAO_REQ.HAT
 ..Download finished
 ..Upload data to current experiment
 ..Requi

# Plots

In [14]:
runner.draw_training_history()

-------------------- draw training history -------------------
Not training a model. Downloading plot from Neptune
 ..Experiment ID: ICAO-338
 ..Downloading plot from previous experiment
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-338
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
 ..Download finished
 ..Uploading plot
 ..Upload finished
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 12 operations to synchronize with Neptune. Do not kill this process.
All 12 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-338


# Load Best Model

In [15]:
runner.load_best_model()

-------------------- load best model -------------------
..Loading best model
..Model loaded
...Model path: prev_trained_models/multi_task/fvc_not_aligned/multi_reqs/ICAO-338


# Saving Trained Model

In [16]:
runner.save_model()

-------------------- save model -------------------
Not saving model!


# Test Trained Model

## Validation Split

In [17]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

Testing Trained Model
Predicting labels....
18/18 [==============================] - 37s 2s/step
Prediction finished!


/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:633: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
Error occurred during asynchronous operation processing: Value of float attribute cannot be malformed.
/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:803: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


  Task  0: mouth           | EER_mean: 0.04520 | EER_interp: 0.04230 | ACC: 0.95610
  Task  1: rotation        | EER_mean: 0.18980 | EER_interp: 0.19490 | ACC: 0.80260
  Task  2: l_away          | EER_mean: 0.10300 | EER_interp: 0.10640 | ACC: 0.89210
  Task  3: eyes_closed     | EER_mean: 0.02830 | EER_interp: 0.02550 | ACC: 0.97620
  Task  4: close           | EER_mean: 0.16670 | EER_interp: 0.14060 | ACC: 0.88480
  Task  5: hat             | EER_mean: 0.01080 | EER_interp: 0.00870 | ACC: 0.99270
  Task  6: dark_glasses    | EER_mean: 0.01920 | EER_interp: 0.01060 | ACC: 0.99630
  Task  7: frames_heavy    | EER_mean: 0.02020 | EER_interp: 0.00000 | ACC: 0.97990
  Task  8: frame_eyes      | EER_mean: 0.03540 | EER_interp: 0.03420 | ACC: 0.96530
  Task  9: flash_lenses    | EER_mean: 0.05530 | EER_interp: 0.05420 | ACC: 0.94520
  Task 10: veil            | EER_mean: 0.00000 | EER_interp: 0.00000 | ACC: 0.93970
  Task 11: reflection      | EER_mean: 0.13580 | EER_interp: 0.13880 | ACC: 

## Test Split

In [18]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

Testing Trained Model
Predicting labels....
3/9 [=========>....................] - ETA: 8s

Error occurred during asynchronous operation processing: Value of float attribute cannot be malformed.


9/9 [==============================] - 17s 2s/step
Prediction finished!


Error occurred during asynchronous operation processing: Value of float attribute cannot be malformed.


  Task  0: mouth           | EER_mean: 0.05480 | EER_interp: 0.03670 | ACC: 0.97220
  Task  1: rotation        | EER_mean: 0.18750 | EER_interp: 0.18750 | ACC: 0.81250
  Task  2: l_away          | EER_mean: 0.11490 | EER_interp: 0.11960 | ACC: 0.87850
  Task  3: eyes_closed     | EER_mean: 0.03860 | EER_interp: 0.03750 | ACC: 0.96180
  Task  4: close           | EER_mean: 0.11190 | EER_interp: 0.00000 | ACC: 0.88890
  Task  5: hat             | EER_mean: 0.01960 | EER_interp: 0.01610 | ACC: 0.98610
  Task  6: dark_glasses    | EER_mean: 0.01950 | EER_interp: 0.00000 | ACC: 0.98260
  Task  7: frames_heavy    | EER_mean: 0.00000 | EER_interp: 0.00000 | ACC: 0.99310
  Task  8: frame_eyes      | EER_mean: 0.04900 | EER_interp: 0.04830 | ACC: 0.95140
  Task  9: flash_lenses    | EER_mean: 0.04390 | EER_interp: 0.03860 | ACC: 0.95830
  Task 10: veil            | EER_mean: 0.04760 | EER_interp: 0.04820 | ACC: 0.95140
  Task 11: reflection      | EER_mean: 0.13460 | EER_interp: 0.13170 | ACC: 

# Visualize Model Classification

# Finishing Experiment Manager

In [19]:
runner.finish_experiment()

-------------------- finish experiment -------------------
Finishing Neptune
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 146 operations to synchronize with Neptune. Do not kill this process.


Error occurred during asynchronous operation processing: Value of float attribute cannot be malformed.
Error occurred during asynchronous operation processing: Value of float attribute cannot be malformed.


All 146 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-349
